<a href="https://colab.research.google.com/github/UDICatNCHU/PyTorch-Tutorial/blob/master/Lesson_7_BERT_on_%E5%A4%96%E8%B3%A3%E8%B3%87%E6%96%99%E9%9B%86_verbose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **環境設定**

### **取得 google drive 存取權限**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [1]:
!wget https://www.dropbox.com/s/ayihrt5n8kv4ofx/waimai_10k_zh_tw.csv?dl=0

--2020-11-13 09:14:16--  https://www.dropbox.com/s/ayihrt5n8kv4ofx/waimai_10k_zh_tw.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ayihrt5n8kv4ofx/waimai_10k_zh_tw.csv [following]
--2020-11-13 09:14:17--  https://www.dropbox.com/s/raw/ayihrt5n8kv4ofx/waimai_10k_zh_tw.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbcbb21cfd07337ba70735ea338.dl.dropboxusercontent.com/cd/0/inline/BDEe-daQpaadGDt5xPe_YibfYSbeLwQMUtY2G_i5SMS9XhapoMSk-941bpDBQ0_TaO_tWtYWrfS7UsyWegnaTWZSrY8Q080X4XCoJUU-3e2PYESAj3lDq7EQqN0ToOF1b4E/file# [following]
--2020-11-13 09:14:17--  https://ucbcbb21cfd07337ba70735ea338.dl.dropboxusercontent.com/cd/0/inline/BDEe-daQpaadGDt5xPe_YibfYSbeLwQMUtY2G_i5SMS9XhapoMSk-941bpDBQ0_TaO_tWtYWrfS7UsyWegnaT

In [2]:
!mv 'waimai_10k_zh_tw.csv?dl=0' waimai_10k_zh_tw.csv

In [3]:
!head waimai_10k_zh_tw.csv

label,review
1,很快，好吃，味道足，量大
1,沒有送水沒有送水沒有送水
1,非常快，態度好。
1,方便，快捷，味道可口，快遞給力
1,菜味道很棒！送餐很及時！
1,今天師傅是不是手抖了，微辣格外辣！
1,"送餐快,態度也特別好,辛苦啦謝謝"
1,超級快就送到了，這麼冷的天氣騎士們辛苦了。謝謝你們。麻辣香鍋依然很好吃。
1,經過上次晚了2小時，這次超級快，20分鐘就送到了……


In [4]:
!wc waimai_10k_zh_tw.csv

 11988  11988 919380 waimai_10k_zh_tw.csv


### **切 training data 及 testing data**


In [5]:
# 將要訓練的句子存起來
def SaveSentence(filepath, sent_list):
    f = open(filepath, 'w', encoding='UTF-8')
    for sent in sent_list:
        f.write(sent + '\n')
    f.close()

# 分割train_data和test_data
def data_Split(FileName):

    fp = open(FileName, 'r', encoding='utf-8')
    line = fp.readline()        # 第一行是label,review
    line = fp.readline()

    train_sent_num = 3000
    test_sent_num = 1000
    # 計算目前資料筆數
    train_positive_num = 0 
    train_negative_num = 0
    test_positive_num = 0
    test_negative_num = 0
    train_data = []
    test_data = []

    # 用 while 逐行讀取檔案內容，直至檔案結尾
    while line:
        sent = ''
        sent = line.replace('\n', '')

        if line[:2] == '1,':
            if train_positive_num < train_sent_num:
                train_data.append(sent)
                train_positive_num += 1
            elif test_positive_num < test_sent_num:
                test_data.append(sent)
                test_positive_num += 1
        else:
            if train_negative_num < train_sent_num:
                train_data.append(sent)
                train_negative_num += 1
            elif test_negative_num < test_sent_num:
                test_data.append(sent)
                test_negative_num += 1
        
        line = fp.readline()
    
    fp.close()

    SaveSentence('train_data.txt', train_data)
    SaveSentence('test_data.txt', test_data)

In [6]:
data_Split('waimai_10k_zh_tw.csv')

In [7]:
!wc train_data.txt

  6000   6000 430529 train_data.txt


### **安裝所需的函式庫**

In [8]:
!pip install transformers

In [ ]:
%tensorflow_version 2.x

In [17]:
import os
import pickle
import torch
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

In [9]:
!wget https://www.dropbox.com/s/fxa234s3de7k55q/bert-base-chinese-vocab.txt?dl=0

--2020-11-13 09:14:37--  https://www.dropbox.com/s/fxa234s3de7k55q/bert-base-chinese-vocab.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fxa234s3de7k55q/bert-base-chinese-vocab.txt [following]
--2020-11-13 09:14:37--  https://www.dropbox.com/s/raw/fxa234s3de7k55q/bert-base-chinese-vocab.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0bd446a43b303b5f614a4aaf55.dl.dropboxusercontent.com/cd/0/inline/BDFlxMcPJ3UF1PQe_EHyElwCVfMSvmj9cRlyXbxT_NMv9CGRqjF8cGU4rstkgREluHOs6SKLVX22SBReh0Hz5nDiOExqPn9mN5ocTWoeU-3n16hNfSqlRb4lIfs-UrOGCvA/file# [following]
--2020-11-13 09:14:38--  https://uc0bd446a43b303b5f614a4aaf55.dl.dropboxusercontent.com/cd/0/inline/BDFlxMcPJ3UF1PQe_EHyElwCVfMSvmj9cRlyXbxT_NMv9CGRqjF8cGU4rstkgREl

In [10]:
!mv 'bert-base-chinese-vocab.txt?dl=0' bert-base-chinese-vocab.txt

In [11]:
!head bert-base-chinese-vocab.txt

[PAD]
[unused1]
[unused2]
[unused3]
[unused4]
[unused5]
[unused6]
[unused7]
[unused8]
[unused9]


### **將資料轉換成輸入格式**

In [12]:
def convert_data_to_feature(FileName):
    # 載入字典
    tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')

    # 載入資料
    Labels = []
    Sentences = []
    with open(FileName,'r',encoding='utf-8') as f:
        data = f.read()
    LS_pairs = data.split("\n")

    for LS_pair in LS_pairs:
        if LS_pair != "":
            try:
                L = LS_pair[:1]
                S = LS_pair[2:]
                Labels.append(int(L))
                Sentences.append(S)
            except:
                continue
    
    assert len(Labels) == len(Sentences)

    # BERT input embedding
    max_seq_len = 0     # 紀錄最大長度
    input_ids = []
    original_length = []    # 紀錄原本長度
    for S in Sentences:
        # 將句子切割成一個個token
        word_piece_list = tokenizer.tokenize(S)
        # 將token轉成字典中的id
        input_id = tokenizer.convert_tokens_to_ids(word_piece_list)
        # 補上[CLS]和[SEP]
        input_id = tokenizer.build_inputs_with_special_tokens(input_id)

        if(len(input_id)>max_seq_len):
            max_seq_len = len(input_id)
        input_ids.append(input_id)

    print("最長句子長度:",max_seq_len)
    assert max_seq_len <= 512 # 小於BERT-base長度限制
    max_seq_len = 512

    # 補齊長度
    for c in input_ids:
        # 紀錄原本長度
        length = len(c)
        original_length.append(length)
        while len(c)<max_seq_len:
            c.append(0)
    
    token_type_ids = [[0]*max_seq_len for i in range(len(Sentences))]         # token_type_ids # 儲存的是句子的id，id為0就是第一句，id為1就是第二句
    attention_mask = []                                                      # attention_mask # 1代表是真實的單詞id，0代表補齊長度
    for i in range(len(Sentences)):
        attention_id = []
        for j in range(original_length[i]):
            attention_id.append(1)
        while len(attention_id)<max_seq_len:
            attention_id.append(0)
        attention_mask.append(attention_id)

    assert len(input_ids) == len(token_type_ids) and len(input_ids) == len(attention_mask) and len(input_ids) == len(Labels)

    data_features = {'input_ids':input_ids,
                    'token_type_ids':token_type_ids,
                    'attention_mask':attention_mask,
                    'labels':Labels}

    return data_features

### **輸入格式轉Dataset**

In [15]:
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    token_type_ids = data_feature['token_type_ids']
    attention_mask = data_feature['attention_mask']
    labels = data_feature['labels']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_token_type_ids = torch.tensor([token_type_id for token_type_id in token_type_ids], dtype=torch.long)
    all_attention_mask_ids = torch.tensor([attention_id for attention_id in attention_mask], dtype=torch.long)
    all_labels = torch.tensor([label for label in labels], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_token_type_ids, all_attention_mask_ids, all_labels)

    return dataset

In [18]:
tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
data_features = convert_data_to_feature('train_data.txt')

print(data_features['input_ids'][5999])
print(tokenizer.convert_ids_to_tokens(data_features['input_ids'][5999]))
print(data_features['token_type_ids'][5999])
print(data_features['attention_mask'][5999])
print(data_features['labels'][5999])

最長句子長度: 436
[101, 7618, 2094, 6843, 6882, 889, 6963, 3893, 749, 8024, 2218, 1391, 749, 1060, 1366, 134, 134, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### **Fine-Tuning**

In [22]:
# 計算正確值
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [23]:
# set device
device = torch.device('cuda')

train_data_feature = convert_data_to_feature('train_data.txt')
test_data_feature = convert_data_to_feature('test_data.txt')
train_dataset = makeDataset(train_data_feature)
test_dataset = makeDataset(test_data_feature)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

model = BertForSequenceClassification.from_pretrained('bert-base-chinese')
model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#     ]
Learning_rate = 5e-6       # 學習率
optimizer = AdamW(model.parameters(), lr=Learning_rate, eps=1e-8)

for epoch in range(3):
    # 訓練模式
    model.train()
    All_train_correct = 0.0
    AllTrainLoss = 0.0
    count = 0
    for batch_dict in train_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)

        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],             
            attention_mask = batch_dict[2],             
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]
            
        train_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_train_correct += train_correct
        AllTrainLoss += loss.item()
        count += 1

        model.zero_grad()
        loss.backward()
        optimizer.step()
            
    Average_train_correct = round(All_train_correct/count, 3)
    Average_train_loss = round(AllTrainLoss/count, 3)

    # 測試模式
    model.eval()
    All_test_correct = 0.0
    AllTestLoss = 0.0
    count = 0
    for batch_dict in test_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)

        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],            
            attention_mask = batch_dict[2],           
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]

        test_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_test_correct += test_correct
        AllTestLoss += loss.item()

        count += 1
        
    Average_test_correct = round(All_test_correct/count, 3)
    Average_test_loss = round(AllTestLoss/count, 3)

    print('第' + str(epoch+1) + '次' + '訓練模式，loss為:' + str(Average_train_loss) + ' 正確率為' + str(Average_train_correct)+ '，測試模式，loss為:' + str(Average_test_loss) + ' 正確率為' + str(Average_test_correct))
    
# 模型存檔
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained('trained_model')

最長句子長度: 436
最長句子長度: 307


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: ignored

### **測試**

In [ ]:
def to_input_id(sentence_input):
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input)))

In [ ]:
# load and init
tokenizer = BertTokenizer(vocab_file='bert-base-chinese-vocab.txt')
config = BertConfig.from_pretrained('trained_model/config.json')
model = BertForSequenceClassification.from_pretrained('trained_model/pytorch_model.bin', from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
model.eval()

print('請輸入句子')
sentence = input()

input_id = to_input_id(sentence)
assert len(input_id) <= 512

while len(input_id)<512:
    input_id.append(0)

input_ids = torch.LongTensor(input_id).unsqueeze(0)

# predict時，因為沒有label所以沒有loss
outputs = model(input_ids)
predicts = outputs[:2]
predicts = predicts[0]
max_val = torch.max(predicts)
predict_label = (predicts == max_val).nonzero().numpy()[0][1] # 在第1維度取最大值並返回索引值 

if str(predict_label) == '1':
    print('正面')
else:
    print('負面')

請輸入句子
好吃，很喜歡店面的裝飾，很棒的小店。
正面
